In [ ]:
"""
BERECHNUNG DER NORMALISIERTEN AOPC (NAOPC) MITTELS BEAM SEARCH

Dieses Skript implementiert die Berechnung des "Normalized Area Over the Perturbation Curve" (NAOPC).
Hier wird der "Beam Search"-Algorithmus implementiert.

Der Prozess umfasst folgende Schritte:
1.  **Laden von Modell und Daten:** Ein vortrainiertes neuronales Netz und der zugehörige Datensatz werden
    geladen.
2.  **Ermittlung der Ober- und Untergrenze mit Beam Search:**
    * **Upper Bound (Most-Relevant First):** Der Beam-Search-Algorithmus wird eingesetzt, um eine
        nahezu optimale Reihenfolge für die Störung (Perturbation) der **wichtigsten** Bildregionen zu finden.
        Dies führt zur maximal möglichen Reduktion der Modellgenauigkeit und definiert die Obergrenze des AOPC-Wertes.
    * **Lower Bound (Least-Relevant First):** Analog dazu wird die Reihenfolge für die Störung der
        **unwichtigsten** Bildregionen gesucht. Dies führt zur geringstmöglichen Reduktion der
        Modellgenauigkeit und definiert die Untergrenze.
3.  **Berechnung des AOPC für XAI-Methoden:** Für jede zu evaluierende XAI-Methode (z.B. LRP, Gradient)
    wird der Standard-AOPC-Wert durch Perturbation gemäß der von ihr erzeugten Relevanz-Heatmap berechnet.
4.  **Normalisierung zum NAOPC-Score:** Der AOPC-Wert jeder XAI-Methode wird anschließend anhand der zuvor
    ermittelten Ober- und Untergrenzen normalisiert. Der resultierende NAOPC-Score liegt zwischen 0 und 1.
"""

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from tqdm.notebook import tqdm
import os
import multiprocessing
from functools import partial

# Lade model
model_path = './models/VGG16 MODELS/vgg16_cifar10_a87_va85_l378_vl448.h5'
# Alle Parameter die man braucht
# Number of images to test from the dataset
num_images = 800
BEAM_WIDTH = 1 # 2,4,8,16
# Batch size for processing images during the search
# A smaller batch size uses less VRAM but may be slower. Adjust based on your GPU.
BATCH_SIZE = 100 #64 is save
# Image dimensions your model expects
IMG_HEIGHT = 224
IMG_WIDTH = 224
# Perturbation settings
GRID_SIZE = 4  # We will have a 4x4 grid of regions
NUM_STEPS = GRID_SIZE * GRID_SIZE  # 16 perturbation steps
REGION_HEIGHT = IMG_HEIGHT // GRID_SIZE
REGION_WIDTH = IMG_WIDTH // GRID_SIZE


# Dataset Loading
ds_test_list, ds_info = tfds.load(
    'cifar10',
    split=['test'],
    shuffle_files=False,
    as_supervised=True,
    with_info=True,
    data_dir='/mnt/data/datasets'
)
ds_test = ds_test_list[0]

def preprocess_vgg(image, label):
    # Resize to VGG16's expected input size
    image = tf.image.resize(image, [224, 224])
    # Apply VGG16 preprocessing (convert to float32 + mean subtraction)
    image = preprocess_input(image)
    return image, label

ds_test = ds_test.map(preprocess_vgg, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.take(num_images)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)


print(f"Loaded {num_images} images, batched into sizes of {BATCH_SIZE}.")
model = tf.keras.models.load_model(model_path, compile=False)

2025-07-15 14:16:44.653709: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-07-15 14:16:44.653884: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-07-15 14:16:45.121107: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-15 14:16:46.054700: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-15 14:16:55.659596: W tensorflow/compiler/tf2

Loaded 800 images, batched into sizes of 100.


In [ ]:
# Perturbation und Beam Search logik mit worker
# Import the worker function from the separate .py file
from worker import generate_candidates_for_beam

def get_regions():
    """Calculates the coordinates for each of the 16 regions."""
    regions = []
    for i in range(GRID_SIZE):
        for j in range(GRID_SIZE):
            y1 = i * REGION_HEIGHT
            y2 = y1 + REGION_HEIGHT
            x1 = j * REGION_WIDTH
            x2 = x1 + REGION_WIDTH
            regions.append((y1, y2, x1, x2))
    return regions

def get_true_class_probabilities(predictions, true_labels):
    """Extracts the model's confidence in the true class from a prediction matrix."""
    batch_indices = tf.range(tf.shape(true_labels)[0], dtype=tf.int64)
    gather_indices = tf.stack([batch_indices, tf.cast(true_labels, tf.int64)], axis=1)
    return tf.gather_nd(predictions, gather_indices)

def run_parallel_beam_search(model, image_batch, label_batch, regions, pool, mode, beam_width=4):
    """
    Performs a parallelized beam search using a provided multiprocessing pool.
    """
    if mode not in ['upper', 'lower']:
        raise ValueError("Mode must be 'upper' or 'lower'")
    num_parallel_images = tf.shape(image_batch)[0]
    num_regions = len(regions)

    base_predictions = model.predict(image_batch, verbose=0)
    base_probs = get_true_class_probabilities(base_predictions, label_batch)
    initial_log_scores = tf.math.log(base_probs + 1e-9)
    beams = [[(initial_log_scores[i], [], image_batch[i])] for i in range(num_parallel_images)]
    prob_traces = [[base_probs[i].numpy()] for i in range(num_parallel_images)]

    for k in tqdm(range(num_regions), desc=f"Beam Search ({mode})", leave=False, ncols=100):
        items_to_process = []
        for i in range(num_parallel_images):
            for score, seq, img_state in beams[i]:
                items_to_process.append((i, score, seq, img_state))
        print(f"  [Step {k+1}] Starting with {len(items_to_process)} active beam paths.")
        if not items_to_process:
            break

        worker_func = partial(generate_candidates_for_beam, regions=regions, num_regions=num_regions)
        results_nested = pool.map(worker_func, items_to_process)

        all_candidates_for_this_step = []
        candidate_map = []
        for sublist in results_nested:
            for perturbed_image, map_info in sublist:
                all_candidates_for_this_step.append(perturbed_image)
                candidate_map.append(map_info)

        if not all_candidates_for_this_step:
            break

        print(f"  [Step {k+1}] Generated {len(all_candidates_for_this_step)} new candidates.")

        candidate_images_tensor = tf.stack(all_candidates_for_this_step)
        candidate_labels_list = [label_batch[info['image_idx']] for info in candidate_map]
        candidate_labels_tensor = tf.stack(candidate_labels_list)
        predictions = model.predict(candidate_images_tensor, verbose=0)
        candidate_probs = get_true_class_probabilities(predictions, candidate_labels_tensor)
        candidate_log_probs = tf.math.log(candidate_probs + 1e-9)

        new_beams = [[] for _ in range(num_parallel_images)]
        for j, info in enumerate(candidate_map):
            image_idx, new_score = info['image_idx'], info['prev_score'] + candidate_log_probs[j]
            new_sequence, new_image_state = info['sequence'], all_candidates_for_this_step[j]
            new_beams[image_idx].append((new_score, new_sequence, new_image_state))

        for i in range(num_parallel_images):
            if not new_beams[i]:
                new_beams[i] = beams[i]

            reverse_sort = (mode == 'lower')
            sorted_paths = sorted(new_beams[i], key=lambda x: x[0], reverse=reverse_sort)
            beams[i] = sorted_paths[:beam_width]

            if beams[i]:
                prob_traces[i].append(tf.math.exp(beams[i][0][0]).numpy())
            else:
                prob_traces[i].append(prob_traces[i][-1] if prob_traces[i] else 0.0)

    return tf.convert_to_tensor(prob_traces, dtype=tf.float32)

def calculate_aopc(prob_sequences):
    """Calculates the Area Over the Perturbation Curve."""
    base_probs = prob_sequences[:, 0:1]
    step_probs = prob_sequences[:, 1:]
    aopc = tf.reduce_mean(tf.reduce_sum(base_probs - step_probs, axis=1))
    return aopc

In [ ]:
# Hauptlogik, Berechnung, Ouput

def main():
    """Contains the main logic of the script to be run."""
    regions = get_regions()

    print("- --Starting Main Execution ---")

    all_upper_bound_sequences = []
    all_lower_bound_sequences = []

    # Erstelle multiprocessing Pool, passt automatisch an menge an kerne der CPU an.
    with multiprocessing.Pool(processes=os.cpu_count()) as pool:
        for image_batch, label_batch in tqdm(ds_test, desc="Processing Batches"):

            # Pass the pool object to the search function
            upper_bound_seqs = run_parallel_beam_search(model, image_batch, label_batch, regions, pool, mode='upper', beam_width=BEAM_WIDTH)
            all_upper_bound_sequences.append(upper_bound_seqs)

            lower_bound_seqs = run_parallel_beam_search(model, image_batch, label_batch, regions, pool, mode='lower', beam_width=BEAM_WIDTH)
            all_lower_bound_sequences.append(lower_bound_seqs)

    # Berechnung der unteren und oberen grenze
    all_upper_bound_sequences = tf.concat(all_upper_bound_sequences, axis=0)
    all_lower_bound_sequences = tf.concat(all_lower_bound_sequences, axis=0)

    aopc_upper = calculate_aopc(all_upper_bound_sequences)
    aopc_lower = calculate_aopc(all_lower_bound_sequences)

    print("\n Results")
    print(f"Processed {all_upper_bound_sequences.shape[0]} images with {NUM_STEPS} perturbation steps.")
    print(f"AOPC Upper Bound (Most disruptive): {aopc_upper.numpy():.4f}")
    print(f"AOPC Lower Bound (Least disruptive): {aopc_lower.numpy():.4f}")

    Normalized_AOPC_upper = aopc_upper / NUM_STEPS
    Normalized_AOPC_lower = aopc_lower / NUM_STEPS

    print("\n--- Normalized AOPC Values ---")
    print(f"Normalized AOPC Upper Bound: {Normalized_AOPC_upper.numpy():.4f}")
    print(f"Normalized AOPC Lower Bound: {Normalized_AOPC_lower.numpy():.4f}")

if __name__ == '__main__':
    # spawn für cuda kompatibilät
    try:
        multiprocessing.set_start_method('spawn', force=True)
    except RuntimeError as e:
        print(f"Multiprocessing context already set: {e}")
        pass

    main()

In [ ]:
# In diesem teil müssen berechnet AOPC werte in einem dict format übergeben werden.
Normalized_AOPC_upper = Normalized_AOPC_upper / NUM_STEPS
Normalized_AOPC_lower = Normalized_AOPC_lower / NUM_STEPS

print(f"Normalized AOPC Upper Bound: {Normalized_AOPC_upper:.4f}")
print(f"Normalized AOPC Lower Bound: {Normalized_AOPC_lower:.4f}")

user_aopc_dict = {
    "random": 0.624,
    "gradient": 0.632,
    "smoothgrad": 0.631,
    "deconvnet": 0.576,
    "guided_backprop": 0.608,
    "deep_taylor_bounded": 0.637,
    "input_x_gradient": 0.650,
    "integrated_gradient": 0.644,
    "lrp_Z": 0.649,
    "lrp_E": 0.649
}

print("Normalizing AOPC Scores by Method")
print(f"{'Method':<25} | {'Normalized NAOPC':<20}")
print(f"{'-'*25:<25} | {'-'*20:<20}")


test = Normalized_AOPC_upper - Normalized_AOPC_lower
for method_name, aopc_val in user_aopc_dict.items():
    # NAOPC Formel
    naopc_score = (aopc_val - Normalized_AOPC_lower) / test
    #print("test:",aopc_val)
    #print(f"{naopc_score:<20.4f}")
    formatted_score = f"{naopc_score:.3f}".replace('.', ',')
    print(f"{formatted_score:<20}")